In [ ]:
import torch
#import huggingface as hf

# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-koxbu4xe/unsloth_3c1c04c7dcde42f8a188592f206a6737
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-koxbu4xe/unsloth_3c1c04c7dcde42f8a188592f206a6737
  Resolved https://github.com/unslothai/unsloth.git to commit 92dce38e8b3c1db209cef860d90b60188e95f0f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
torch.__version__

'2.3.0+cu121'

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 42,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Data Prep

In [ ]:
import pandas as pd
import numpy as np

train = pd.read_csv("/content/drive/MyDrive/science_exam/llm_Science_exam/dataset/train.csv")
test = pd.read_csv("/content/drive/MyDrive/science_exam/llm_Science_exam/dataset/test.csv")

In [ ]:
import re
from datasets import load_dataset

_prompt = """Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
{}

### Options:
A: {}
B: {}
C: {}
D: {}
E: {}

### Response:
{}"""

# EOS_TOKEN from the tokenizer
EOS_TOKEN = tokenizer.eos_token

# Function to format your DataFrame rows into the required format
def formatting_prompts_func(df):
    df['text'] = df.apply(lambda row: _prompt.format(
        row['prompt'],
        row['A'], row['B'], row['C'], row['D'], row['E'],
        row['answer']
    ) + EOS_TOKEN, axis=1)
    return df

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
formatted_df = formatting_prompts_func(train)
print(formatted_df['text'][0])

Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?

### Options:
A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
D: MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(formatted_df)

In [ ]:
print(train_dataset.column_names)


['id', 'prompt', 'A', 'B', 'C', 'D', 'E', 'answer', 'text']


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
import gc
gc.collect()

38

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 200 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.255100
2,1.366100
3,1.322200
4,1.106700
5,1.324300
6,0.796400
7,0.842900
8,0.658600
9,0.797000
10,1.309100


# Inference

In [ ]:
import re
from datasets import load_dataset

_prompt = """Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
{}

### Options:
A: {}
B: {}
C: {}
D: {}
E: {}

### Response:

"""

# EOS_TOKEN from the tokenizer
EOS_TOKEN = tokenizer.eos_token

# Function to format your DataFrame rows into the required format
def formatting_prompts_func(df):
    df['text'] = df.apply(lambda row: _prompt.format(
        row['prompt'],
        row['A'], row['B'], row['C'], row['D'], row['E']
    ) , axis=1)
    return df

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

In [ ]:
test.head()

,id,prompt,A,B,C,D,E,text
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,Answer the following multiple choice question ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Answer the following multiple choice question ...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,Answer the following multiple choice question ...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Answer the following multiple choice question ...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,Answer the following multiple choice question ...


In [ ]:
formatted_test = formatting_prompts_func(test)
print(formatted_test['text'][0])

Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?

### Options:
A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
D: MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy

In [ ]:
from transformers import TextStreamer
FastLanguageModel.for_inference(model)

for i in range(10):
  inputs = tokenizer(
  [formatted_test["text"][i]], return_tensors = "pt").to("cuda")
  text_streamer = TextStreamer(tokenizer)
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 4)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?

### Options:
A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
D: MOND is a theory that reduces the discrepancy between the observed missing baryon

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
Which of the following is an accurate definition of dynamic scaling in self-similar systems?

### Options:
A: Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times exhibits similarity to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.
B: Dynamic scaling refers to the non-evolution of self-similar systems, where data obtained from snapshots at fixed times is similar to the respective data taken from snapshots of any earlier or later time. This similarity is tested by a certain time-dependent stochastic variable x.
C: Dynamic scaling refers to the evolution of self-similar systems, where data obtained from snapshots at fixed times is dissimilar to the respective data tak

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

<|begin_of_text|>Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
Which of the following statements accurately describes the blocking temperature of an antiferromagnetic layer in a spin valve?

### Options:
A: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature at which the magnetization of the ferromagnetic layer becomes aligned with the magnetic field. The blocking temperature is typically higher than the Néel temperature.
B: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature below which the layer loses its ability to "pin" the magnetization direction of an adjacent ferromagnetic layer. The blocking temperature is typically higher than the Néel temperature.
C: The blocking temperature of an antiferromagnetic layer in a spin valve is the temperature at which the ferromagnetic layer becomes completely demagnetized. 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
What is the term used in astrophysics to describe light-matter interactions resulting in energy shifts in the radiation field?

### Options:
A: Blueshifting
B: Redshifting
C: Reddening
D: Whitening
E: Yellowing

### Response:

C<|end_of_text|>


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Answer the following multiple choice question by giving the most appropriate response. Answer should be one among [A, B, C, D, E]

### Question:
What is the role of axioms in a formal theory?

### Options:
A: Basis statements called axioms form the foundation of a formal theory and, together with the deducing rules, help in deriving a set of statements called theorems using proof theory.
B: Axioms are supplementary statements added to a formal theory that break down otherwise complex statements into more simple ones.
C: Axioms are redundant statements that can be derived from other statements in a formal theory, providing additional perspective to theorems derived from the theory.
D: The axioms in a theory are used for experimental validation of the theorems derived from the statements in the theory.
E: The axioms in a formal theory are added to prove that the statements derived from the theory are true, irrespective of their validity in the real world.

### Response:
